<a href="https://colab.research.google.com/github/esthy13/cil-intrusion-detection/blob/main/notebooks/Cybersecurity_icarl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import copy
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import glob
import random

In [ ]:
def set_seed(seed=42):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

set_seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device


device(type='cpu')

In [ ]:
class IDSIncrementalDataset(Dataset):
    def __init__(self, df, feature_cols, label_col, label_to_id, indices=None):
        self.features = df[feature_cols].values.astype(np.float32)
        self.labels = df[label_col].map(label_to_id).values.astype(np.int64)
        self.label_to_id = label_to_id

        if indices is None:
            self.indices = np.arange(len(self.labels))
        else:
            self.indices = indices

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        x = torch.tensor(self.features[idx])
        y = torch.tensor(self.labels[idx])
        return x, y

    def filter_by_classes(self, class_list):
        mask = np.isin(self.labels, class_list)

        new_dataset = IDSIncrementalDataset.__new__(IDSIncrementalDataset)

        new_dataset.features = self.features[mask]
        new_dataset.labels = self.labels[mask]
        new_dataset.label_to_id = self.label_to_id
        new_dataset.indices = self.indices[mask]

        return new_dataset

In [ ]:
gghhhh

In [ ]:
class IDSNet(nn.Module):
    def __init__(self, input_dim, feature_dim=128):
        super().__init__()

        self.feature_extractor = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Linear(256, feature_dim)
        )

        self.classifier = None  # SOLO para entrenamiento

    def forward(self, x):
        feats = self.feature_extractor(x)
        feats = F.normalize(feats, dim=1)

        if self.classifier is not None:
            logits = self.classifier(feats)
            return logits, feats

        return feats

    def update_classifier(self, num_classes):
        self.classifier = nn.Linear(
            self.feature_extractor[-1].out_features,
            num_classes
        )

In [ ]:
# Metrics

import torch
import numpy as np
from sklearn.metrics import f1_score, accuracy_score

def accuracy(y_true, y_pred):

  return accuracy_score(y_true, y_pred)

def macro_f1(y_true, y_pred):

  return f1_score(y_true, y_pred, average='macro')

def average_accuracy(task_accuracies):

  """
  task_accuracies: list of accuracies up to current task
  """
  return np.mean(task_accuracies)

def compute_forgetting(results):
    """
    results: 2D list or numpy array
             shape (num_tasks, num_tasks)
             results[t][i] = metric on task i after training task t
    """

    results = np.array(results)
    num_tasks = results.shape[0]

    forgetting_per_task = []

    for t in range(1, num_tasks):
        f_t = 0
        for i in range(t):
            best_past = np.max(results[:t, i])
            current = results[t, i]
            f_t += (best_past - current)

        f_t /= t
        forgetting_per_task.append(f_t)

    return np.mean(forgetting_per_task)

In [ ]:
class ICaRL:
    def __init__(self, model, device, memory_size=2000):
        self.model = model.to(device)
        self.device = device
        self.memory_size = memory_size

        self.seen_classes = []
        self.exemplars = {}      # class_id -> indices
        self.class_means = {}    # class_id -> mean vector

        self.old_model = None

    def distillation_loss(self, old_logits, new_logits, T=2):
        old_probs = F.softmax(old_logits / T, dim=1)
        new_log_probs = F.log_softmax(new_logits / T, dim=1)
        return F.kl_div(new_log_probs, old_probs, reduction="batchmean") * (T * T)

    def add_classes(self, new_classes):
        self.seen_classes += new_classes
        num_classes = len(self.seen_classes)

        self.model.update_classifier(num_classes)

        if self.old_model is not None:
            old_w = self.old_model.classifier.weight.data
            self.model.classifier.weight.data[:old_w.size(0)] = old_w

        self.old_model = copy.deepcopy(self.model).eval()

    def train_task(self, dataset, epochs=10, batch_size=64, lr=1e-3):
        self.model.train()

        loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
        optimizer = torch.optim.Adam(self.model.parameters(), lr=lr)

        for epoch in range(epochs):
            total_loss = 0.0

            for x, y in loader:
                x, y = x.to(self.device), y.to(self.device)

                logits, feats = self.model(x)
                loss = F.cross_entropy(logits, y)

                if self.old_model is not None:
                    with torch.no_grad():
                        old_logits, _ = self.old_model(x)
                    loss += self.distillation_loss(old_logits, logits)

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                total_loss += loss.item()

            print(f"Epoch {epoch+1} | Loss: {total_loss / len(loader):.4f}")

    def build_exemplars(self, dataset):
        m = self.memory_size // len(self.seen_classes)
        self.model.eval()

        for cls in self.seen_classes:
            idxs = np.where(dataset.labels == cls)[0]

            feats = []
            with torch.no_grad():
                for idx in idxs:
                    x, _ = dataset[idx]
                    _, f = self.model(x.unsqueeze(0).to(self.device))
                    feats.append(f.cpu())

            feats = torch.cat(feats, dim=0)
            class_mean = feats.mean(dim=0)

            distances = torch.norm(feats - class_mean, dim=1)
            selected = idxs[torch.argsort(distances)[:m]]

            self.exemplars[cls] = selected

    def compute_class_means(self, dataset):
        self.class_means = {}
        self.model.eval()

        with torch.no_grad():
            for cls, exemplar_idxs in self.exemplars.items():
                feats = []

                for idx in exemplar_idxs:
                    x, _ = dataset[idx]
                    _, f = self.model(x.unsqueeze(0).to(self.device))
                    feats.append(f.cpu())

                feats = torch.cat(feats, dim=0)
                mean = feats.mean(dim=0)
                mean = mean / mean.norm()   # must be normalized

                self.class_means[cls] = mean

    def predict(self, x):
        self.model.eval()

        with torch.no_grad():
            _, feats = self.model(x.to(self.device))
            feats = feats / feats.norm(dim=1, keepdim=True)

            preds = []
            for f in feats:
                dists = {
                    cls: torch.norm(f - mean.to(self.device))
                    for cls, mean in self.class_means.items()
                }
                preds.append(min(dists, key=dists.get))

        return torch.tensor(preds)



In [ ]:
from torch.utils.data import ConcatDataset

In [ ]:
def train_icarl(trainset, testset, input_dim, feature_dim, device, memory_size, T, batch_size, attacks_pattern):

  if sum(attacks_pattern) + 1 != len(np.unique(trainset.labels)):
    raise ValueError("Attacks pattern is inconsistence with total number of attacks")

  benign_class = 0
  attack_classes = list(trainset.label_to_id.values())[1:]

  model = IDSNet(input_dim=input_dim, feature_dim=feature_dim)

  icarl = ICaRL(model, device, memory_size)

  # em
  task_accuracies = []
  task_f1_macro = []
  task_avg_accuracy = []
  task_forgetting = []

  current_index = 0

  # Loop start
  for i, n_new in enumerate(attacks_pattern):
      #generate scenario
      if i == 0:
          # Task 1: include benign + first attack chunk
          new_attacks = [benign_class] + attack_classes[:n_new]
          current_index += n_new
          seen_so_far = new_attacks

      else:
          new_attacks = attack_classes[current_index:current_index+n_new]
          current_index += n_new
          seen_so_far = [benign_class] + attack_classes[:current_index]

      print(f"\nTask {i+1}")
      print(f"New attacks: {new_attacks}")

      #updates clasiffier
      icarl.add_classes(new_attacks)

      newdata = trainset.filter_by_classes(new_attacks)
      if i > 0:
        olddata = trainset.filter_by_classes(seen_so_far) # filter by indices memory buffer
        combined_dataset = ConcatDataset([newdata, olddata])
      else:
        combined_dataset = newdata

      #icarl.train_task(newdata)

      print(f"Seen so far: {seen_so_far}")
      '''datatask = data.filter_by_classes(seen_so_far)

      combined_dataset = ConcatDataset([new_task_dataset, memory_dataset])
      loader = DataLoader(combined_dataset, batch_size=64, shuffle=True)'''


      #training
      '''icarl.train_task(datatask, epochs=1, batch_size=64, lr=1e-3)'''









In [ ]:
hhh

# Read Datasets

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

path = "/content/drive/MyDrive/cybersecurity-CIC-IDS-2017/train"
csv_paths = glob.glob(f"{path}/*.csv")
assert len(csv_paths) > 0, "No CSV files found"
df = pd.concat([pd.read_csv(p) for p in csv_paths], ignore_index=True)


Mounted at /content/drive


In [ ]:
label_to_id = {'benign': 0, 'bot': 1, 'ddos': 2, 'dos': 3, 'ftp-patator': 4, 'heartbleed': 5, 'infiltration': 6, 'portscan': 7, 'ssh-patator': 8, 'web-attack': 9}
Data = IDSIncrementalDataset(df, feature_cols=["min_seg_size_forward", "Active Mean"], label_col="Label", label_to_id=label_to_id)

In [ ]:
Data.label_to_id.values()

dict_values([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [ ]:
model = IDSNet(input_dim=2)

In [ ]:
train_icarl(trainset=Data, testset=None, input_dim=2, feature_dim=128, device='cpu', memory_size=2000, T=2,
            batch_size=32, attacks_pattern=[3,3,3])
#train_icarl(dataset=Data, model=model, device="cpu", memory_size=2000, T=2, attacks_pattern=[3,3,3])


Task 1
New attacks: [0, 1, 2, 3]
Seen so far: [0, 1, 2, 3]

Task 2
New attacks: [4, 5, 6]
Seen so far: [0, 1, 2, 3, 4, 5, 6]

Task 3
New attacks: [7, 8, 9]
Seen so far: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
